In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### 2.11

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat',
                      sep='\n',
                      delimiter='::',
                      header=None,
                      names=['UserID', 'MovieID','Rating','Timestamp'],
                      engine='python')


In [5]:
movies = pd.read_csv('ml-1m/movies.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['MovieID', 'Title','Genres'], 
                     engine='python')

In [6]:
users = pd.read_csv('ml-1m/users.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['UserID', 'Gender','Age', 'Occupation','Zip-code'], 
                     engine='python')

In [7]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
movies.set_index("MovieID")
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies[["Title", "Year"]] = movies.Title.str.extract(r"(.*) \((\d{4})\)")

movies.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [12]:
movies['MovieID'].count()

3883

In [13]:
movies['Year']  = movies['Year'].astype(int)

In [14]:
countByYear = movies.groupby('Year').count()
countByYear.head()

,MovieID,Title,Genres
Year,,,
1919,3,3,3
1920,2,2,2
1921,1,1,1
1922,2,2,2
1923,3,3,3


In [15]:
countByYear = movies.Year.value_counts(sort=True)
countByYear.head()

1996    345
1995    342
1998    337
1997    315
1999    283
Name: Year, dtype: int64

In [16]:
sexRatio = pd.crosstab(users.Gender, users.Age)
sexRatio

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,78,298,558,338,189,146,102
M,144,805,1538,855,361,350,278


In [17]:
allGenresAccurances = pd.Series(np.concatenate(movies.Genres.str.split('|'), axis=0))
allGenresAccurances.columns = ['Genre']
allGenresAccurances.value_counts(sort=True)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Western          68
Fantasy          68
Film-Noir        44
dtype: int64

In [20]:
ratings_by_movie = pd.merge(movies,ratings, left_index = True, right_on="MovieID").groupby('Title').mean()['Rating'].sort_values(ascending=False)
ratings_by_movie = ratings_by_movie.reset_index()
ratings_by_movie.head()

,Title,Rating
0,"Perils of Pauline, The",5.0
1,Beautiful People,5.0
2,"Cup, The (Ph�rpa)",5.0
3,American Pop,5.0
4,Passion in the Desert,5.0


In [22]:
ratings_count = pd.merge(movies,ratings, left_index=True,right_on='MovieID').groupby('Title').count()['Rating'] 
movies_with_100_ratings = pd.Series(ratings_count[ratings_count > 100]).reset_index()

pd.merge(movies_with_100_ratings,ratings_by_movie, on='Title').sort_values('Rating_y', ascending=False).head()

,Title,Rating_x,Rating_y
1369,Popeye,628,4.560510
1692,Strawberry and Chocolate (Fresa y chocolate),2227,4.554558
941,Kansas City,2223,4.524966
959,Kim,657,4.520548
746,Guardian Angel,1783,4.517106


In [36]:
user_ratings = pd.merge(users.set_index('UserID'),ratings, left_index=True, right_on='UserID').set_index('MovieID')
user_ratings.head()

,Gender,Age,Occupation,Zip-code,UserID,Rating,Timestamp
MovieID,,,,,,,
1193,F,1,10,48067,1,5,978300760
661,F,1,10,48067,1,3,978302109
914,F,1,10,48067,1,3,978301968
3408,F,1,10,48067,1,4,978300275
2355,F,1,10,48067,1,5,978824291


In [62]:
user_ratings_movies = pd.merge(user_ratings, movies, left_index=True, right_on='MovieID')
best_movies_by_gender = user_ratings_movies.groupby(['Gender', 'Title']).agg(['mean', 'count'])['Rating']
best_movies_by_gender = best_movies_by_gender[best_movies_by_gender['count'] > 100]
best_movies_by_gender.reset_index().sort_values(by=['mean','Gender'], ascending=False).drop_duplicates(['Gender'])


,Gender,Title,mean,count
162,F,"Close Shave, A",4.644444,180
1450,M,"Godfather, The",4.583333,1740


In [76]:
mean_year_by_age = user_ratings_movies.groupby(['Age']).agg(['mean'])['Year']
np.round(mean_movies_by_age)

,mean
Age,
1,1989.0
18,1990.0
25,1988.0
35,1985.0
45,1984.0
50,1982.0
56,1982.0


In [87]:
user_ratings_movies_female = user_ratings_movies[user_ratings_movies['Gender'] == 'F']
user_ratings_movies_male = user_ratings_movies[user_ratings_movies['Gender'] == 'M']

print(pd.Series(user_ratings_movies_female.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])
print(pd.Series(user_ratings_movies_male.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])

Drama
Comedy
